In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as pgo
import glob

### Create zipcodes to latitude/longitude dataframe

In [2]:
zipcode_df = pd.read_csv("./us-zip-code-latitude-and-longitude.csv")

zipcodes   = list(zipcode_df['Zip'])
latitudes  = list(zipcode_df['Latitude'])
longitudes = list(zipcode_df['Longitude'])

zipdict = dict(zip(zipcodes, list(zip(latitudes, longitudes)))) 

zipcode_df.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483, -94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351, -92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847, -94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337, -85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129, -91.98027"


### Get anomalous data

In [7]:
tran_df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/full_set.xlsx")
tran_df.head()

,Ô..ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,1,0.073529,2020-02-15 00:00:00
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,4,0.279412,2020-02-15 00:05:45
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31


In [8]:
tran_df = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/fam_3_edu_2_CCAvg_4_group.xlsx")

problem_group_text = []
longs = []
lats = []

problem_dict = {0 : "normal", 2 : "Family + Education + CCAvg", 6 : "All other anomalies"}

for index, row in tran_df.iterrows():
    problem_group_text.append(problem_dict[row['problem_group']])
    zipcode = row['ZIP.Code']
    if zipcode in zipdict:
        latitude, longitude = zipdict[zipcode]
    else:
        latitude, longitude = 0, 0 #replace with median later on
    
    lats.append(latitude)
    longs.append(longitude)
    
tran_df['anomaly_class'] = problem_group_text
tran_df['latitude'] = lats
tran_df['longitude'] = longs

In [9]:
tran_df.head(20)

,id,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,...,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time,anomaly_class,latitude,longitude
1,1,25,1,49,91107,4,1.6,1,0,0,...,0,0,0,1938.100070,6,0.073529,2020-02-15 00:00:00,All other anomalies,34.151066,-118.089740
2,2,45,19,34,90089,3,1.5,1,0,0,...,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52,normal,33.786594,-118.298662
3,3,39,15,11,94720,1,1.0,1,0,0,...,0,0,0,2267.739555,6,0.279412,2020-02-15 00:05:45,All other anomalies,37.866825,-122.253582
4,4,35,9,100,94112,1,2.7,2,0,0,...,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38,normal,37.720931,-122.442410
5,5,35,8,45,91330,4,1.0,2,0,0,...,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31,normal,33.786594,-118.298662
6,6,37,13,29,92121,4,0.4,2,155,0,...,0,1,0,1373.889605,6,0.441176,2020-02-15 00:14:24,All other anomalies,32.899996,-117.207220
7,7,53,27,72,91711,2,1.5,2,0,0,...,0,1,0,26.838841,0,0.000000,2020-02-15 00:17:16,normal,34.110009,-117.719730
8,8,50,24,22,93943,1,0.3,3,0,0,...,0,0,1,22.588337,0,0.000000,2020-02-15 00:20:09,normal,36.354330,-121.132928
9,9,35,10,81,90089,3,0.6,2,104,0,...,0,1,0,26.174493,0,0.000000,2020-02-15 00:23:02,normal,33.786594,-118.298662
10,10,34,9,180,93023,1,8.9,3,0,1,...,0,0,0,25.814199,0,0.000000,2020-02-15 00:25:55,normal,34.456957,-119.253070


# Median Latitude (optional step)

In [44]:
median_latitude = np.median(tran_df['latitude'])
median_longitude = np.median(tran_df['longitude'])

lat_diff = []
long_diff = []

for index, row in tran_df.iterrows():
    
    lat_diff.append(row['latitude'] - median_latitude)
    long_diff.append(row['longitude'] - median_latitude)

tran_df['lat_diff'] = lat_diff
tran_df['lot_diff'] = long_diff

In [45]:
tran_df.head()

,id,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,...,CreditCard,responsetime,problem_group,portion,time,anomaly_class,latitude,longitude,lat_diff,lot_diff
1,1,25,1,49,91107,4,1.6,1,0,0,...,0,1938.100070,6,0.073529,2020-02-15 00:00:00,All other anomalies,34.151066,-118.089740,-0.292680,-152.533486
2,2,45,19,34,90089,3,1.5,1,0,0,...,0,25.165190,0,0.000000,2020-02-15 00:02:52,normal,33.786594,-118.298662,-0.657152,-152.742408
3,3,39,15,11,94720,1,1.0,1,0,0,...,0,2267.739555,6,0.279412,2020-02-15 00:05:45,All other anomalies,37.866825,-122.253582,3.423079,-156.697328
4,4,35,9,100,94112,1,2.7,2,0,0,...,0,18.932080,0,0.000000,2020-02-15 00:08:38,normal,37.720931,-122.442410,3.277185,-156.886156
5,5,35,8,45,91330,4,1.0,2,0,0,...,1,25.627543,0,0.000000,2020-02-15 00:11:31,normal,33.786594,-118.298662,-0.657152,-152.742408


In [46]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
new_tran = tran_df.drop(['portion','latitude','longitude','anomaly_class','ZIP.Code','id','time'], axis=1)

In [11]:
new_tran.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group
1,25,1,49,4,1.6,1,0,0,1,0,0,0,1938.100070,6
2,45,19,34,3,1.5,1,0,0,1,0,0,0,25.165190,0
3,39,15,11,1,1.0,1,0,0,0,0,0,0,2267.739555,6
4,35,9,100,1,2.7,2,0,0,0,0,0,0,18.932080,0
5,35,8,45,4,1.0,2,0,0,0,0,0,1,25.627543,0


In [12]:
from sklearn.decomposition import SparsePCA

In [57]:
X = np.array(new_tran)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(new_tran)

In [58]:
pca = PCA()
pca.fit(X_scaled)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [59]:
PCA(copy=True, n_components=None, whiten=False)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [60]:
len(pca.components_)

16

In [63]:
print('Explained Variance Ratio = ', sum(pca.explained_variance_ratio_[: 7]))

Explained Variance Ratio =  0.7446308711769414


# Explore ideal number of dimensions

In [71]:
for i in range(1, 15):
    print('For ', i , 'dimensions, it explained Variance Ratio = ', sum(pca.explained_variance_ratio_[: i]))

For  1 dimensions, it explained Variance Ratio =  0.14817365255947315
For  2 dimensions, it explained Variance Ratio =  0.27777842406148534
For  3 dimensions, it explained Variance Ratio =  0.4005332672809898
For  4 dimensions, it explained Variance Ratio =  0.5201897003196532
For  5 dimensions, it explained Variance Ratio =  0.6086538134385199
For  6 dimensions, it explained Variance Ratio =  0.6807723769837898
For  7 dimensions, it explained Variance Ratio =  0.7446308711769414
For  8 dimensions, it explained Variance Ratio =  0.8073629863148328
For  9 dimensions, it explained Variance Ratio =  0.8642701027759904
For  10 dimensions, it explained Variance Ratio =  0.916689856075791
For  11 dimensions, it explained Variance Ratio =  0.9510239864616334
For  12 dimensions, it explained Variance Ratio =  0.9770756738410208
For  13 dimensions, it explained Variance Ratio =  0.993864633254381
For  14 dimensions, it explained Variance Ratio =  0.9972365164801997


# Make script that introduces errors to data

In [23]:
df_full = pd.read_excel("/Users/andres/Desktop/CSE498/appdynamics/training_csv/full_set.xlsx")
df_full.head()

,Ô..ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard,responsetime,problem_group,portion,time
1,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,1938.100070,1,0.073529,2020-02-15 00:00:00
2,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,25.165190,0,0.000000,2020-02-15 00:02:52
3,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,2267.739555,4,0.279412,2020-02-15 00:05:45
4,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,18.932080,0,0.000000,2020-02-15 00:08:38
5,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,25.627543,0,0.000000,2020-02-15 00:11:31


In [19]:
df = pd.read_csv("../sanitized_output.csv")
df.head()

,Unnamed: 0,eventCompletionTimestamp,eventTimestamp,pickupTimestamp,requestGUID,responseTime,segments.requestExperience,segments.segmentTimestamp,segments.transactionTime,segments.userData.age,...,segments.userData.income,segments.userData.mortgage,segments.userData.online,segments.userData.personal_loan,segments.userData.securities_account,segments.userData.zip_code,segments.userData.﻿id,transactionId,transactionName,userExperience
0,0,2020-02-11T07:47:06-05:00,2020-02-11T07:46:45-05:00,2020-02-11T07:47:07-05:00,83e1da7f-1ab5-4a3c-8f70-19d2196d3a6b,21,NORMAL,2020-02-11T07:46:45-05:00,21,65.0,...,49.0,0.0,1.0,0.0,0.0,90034.0,4999.0,1127926,zip_91107,NORMAL
1,1,2020-02-11T07:47:06-05:00,2020-02-11T07:46:40-05:00,2020-02-11T07:47:07-05:00,8e232187-eb72-4e3c-9e3b-3ffcc8d26f4f,21,NORMAL,2020-02-11T07:46:40-05:00,21,63.0,...,24.0,0.0,0.0,0.0,0.0,93023.0,4998.0,1127926,zip_91107,NORMAL
2,2,2020-02-11T07:47:06-05:00,2020-02-11T07:46:35-05:00,2020-02-11T07:47:06-05:00,c323894c-00de-4e6e-8f98-d8aabca8de48,20,NORMAL,2020-02-11T07:46:35-05:00,20,30.0,...,15.0,85.0,1.0,0.0,0.0,92037.0,4997.0,1127926,zip_91107,NORMAL
3,3,2020-02-11T07:47:06-05:00,2020-02-11T07:46:30-05:00,2020-02-11T07:47:06-05:00,595ec039-3c30-4a49-9ea0-b955d947ce52,21,NORMAL,2020-02-11T07:46:30-05:00,21,29.0,...,40.0,0.0,1.0,0.0,0.0,92697.0,4996.0,1127926,zip_91107,NORMAL
4,4,2020-02-11T07:47:06-05:00,2020-02-11T07:46:25-05:00,2020-02-11T07:47:06-05:00,21140969-c08a-4d87-8541-8f3020094caa,21,NORMAL,2020-02-11T07:46:25-05:00,21,64.0,...,75.0,0.0,1.0,0.0,0.0,94588.0,4995.0,1127926,zip_91107,NORMAL


In [20]:
df.head(15)

,Unnamed: 0,eventCompletionTimestamp,eventTimestamp,pickupTimestamp,requestGUID,responseTime,segments.requestExperience,segments.segmentTimestamp,segments.transactionTime,segments.userData.age,...,segments.userData.income,segments.userData.mortgage,segments.userData.online,segments.userData.personal_loan,segments.userData.securities_account,segments.userData.zip_code,segments.userData.﻿id,transactionId,transactionName,userExperience
0,0,2020-02-11T07:47:06-05:00,2020-02-11T07:46:45-05:00,2020-02-11T07:47:07-05:00,83e1da7f-1ab5-4a3c-8f70-19d2196d3a6b,21,NORMAL,2020-02-11T07:46:45-05:00,21,65.0,...,49.0,0.0,1.0,0.0,0.0,90034.0,4999.0,1127926,zip_91107,NORMAL
1,1,2020-02-11T07:47:06-05:00,2020-02-11T07:46:40-05:00,2020-02-11T07:47:07-05:00,8e232187-eb72-4e3c-9e3b-3ffcc8d26f4f,21,NORMAL,2020-02-11T07:46:40-05:00,21,63.0,...,24.0,0.0,0.0,0.0,0.0,93023.0,4998.0,1127926,zip_91107,NORMAL
2,2,2020-02-11T07:47:06-05:00,2020-02-11T07:46:35-05:00,2020-02-11T07:47:06-05:00,c323894c-00de-4e6e-8f98-d8aabca8de48,20,NORMAL,2020-02-11T07:46:35-05:00,20,30.0,...,15.0,85.0,1.0,0.0,0.0,92037.0,4997.0,1127926,zip_91107,NORMAL
3,3,2020-02-11T07:47:06-05:00,2020-02-11T07:46:30-05:00,2020-02-11T07:47:06-05:00,595ec039-3c30-4a49-9ea0-b955d947ce52,21,NORMAL,2020-02-11T07:46:30-05:00,21,29.0,...,40.0,0.0,1.0,0.0,0.0,92697.0,4996.0,1127926,zip_91107,NORMAL
4,4,2020-02-11T07:47:06-05:00,2020-02-11T07:46:25-05:00,2020-02-11T07:47:06-05:00,21140969-c08a-4d87-8541-8f3020094caa,21,NORMAL,2020-02-11T07:46:25-05:00,21,64.0,...,75.0,0.0,1.0,0.0,0.0,94588.0,4995.0,1127926,zip_91107,NORMAL
5,5,2020-02-11T07:47:06-05:00,2020-02-11T07:46:20-05:00,2020-02-11T07:47:07-05:00,81c7272a-6007-48be-b7e6-014274eeb348,21,NORMAL,2020-02-11T07:46:20-05:00,21,45.0,...,218.0,0.0,1.0,0.0,0.0,91801.0,4994.0,1127926,zip_91107,NORMAL
6,6,2020-02-11T07:46:36-05:00,2020-02-11T07:46:15-05:00,2020-02-11T07:46:37-05:00,9dbd92ee-91e6-4cdc-801c-51ca2bf66e08,20,NORMAL,2020-02-11T07:46:15-05:00,20,30.0,...,13.0,0.0,0.0,0.0,0.0,90037.0,4993.0,1127926,zip_91107,NORMAL
7,7,2020-02-11T07:46:36-05:00,2020-02-11T07:46:10-05:00,2020-02-11T07:46:36-05:00,81f16c62-76e2-47f6-b94c-9464e699f391,20,NORMAL,2020-02-11T07:46:10-05:00,20,51.0,...,92.0,100.0,0.0,0.0,0.0,91330.0,4992.0,1127926,zip_91107,NORMAL
8,8,2020-02-11T07:46:36-05:00,2020-02-11T07:46:05-05:00,2020-02-11T07:46:37-05:00,c53889c0-65cc-4f0e-8071-b715c1261070,21,NORMAL,2020-02-11T07:46:05-05:00,21,55.0,...,58.0,219.0,0.0,0.0,0.0,95023.0,4991.0,1127926,zip_91107,NORMAL
9,9,2020-02-11T07:46:36-05:00,2020-02-11T07:46:00-05:00,2020-02-11T07:46:37-05:00,50258f89-c687-4442-ba6c-fd8f0702b1d7,20,NORMAL,2020-02-11T07:46:00-05:00,20,24.0,...,38.0,0.0,1.0,0.0,0.0,93555.0,4990.0,1127926,zip_91107,NORMAL
